In [74]:
from pandas import read_csv
from sklearn.cluster import MeanShift
import numpy as np
from io import StringIO
import sys
sys.path.append('../..')
from lib import *
from collections import Counter

In [14]:
df = read_csv('checkins.dat','|', skiprows=[1], na_values='                   ').dropna() 
#, , header=1, , index_col='id', skipfooter=2
df.columns = [s.strip() for s in df.columns]
df.head()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [15]:
# 396632
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 1 to 1021964
Data columns (total 6 columns):
   id                    396634 non-null object
 user_id                 396634 non-null float64
 venue_id                396634 non-null float64
     latitude            396634 non-null float64
     longitude           396634 non-null float64
     created_at          396634 non-null object
dtypes: float64(4), object(2)
memory usage: 21.2+ MB


In [36]:
# with open('checkins.dat', 'r') as f: print(f.read(500))
print(open('checkins.dat').read(700))

   id    | user_id | venue_id |     latitude      |     longitude     |     created_at      
---------+---------+----------+-------------------+-------------------+---------------------
  984301 | 2041916 |     5222 |                   |                   | 2012-04-21 17:39:01
  984222 |   15824 |     5222 |        38.8951118 |       -77.0363658 | 2012-04-21 17:43:47
  984315 | 1764391 |     5222 |                   |                   | 2012-04-21 17:37:18
  984234 |   44652 |     5222 |         33.800745 |         -84.41052 | 2012-04-21 17:43:43
  984249 | 2146840 |     5222 |                   |                   | 2012-04-21 17:42:58
  984268 | 2146843 |     5222 |                   |   


In [47]:
X = df[['latitude','longitude']].sample(100000)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 386679 to 116162
Data columns (total 2 columns):
latitude     100000 non-null float64
longitude    100000 non-null float64
dtypes: float64(2)
memory usage: 2.3 MB


In [48]:
ms = MeanShift(bandwidth=0.1, n_jobs=-1)
%time ms.fit(X)

CPU times: user 3min 36s, sys: 260 ms, total: 3min 37s
Wall time: 3min 37s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [49]:
# labels = ms.labels_
# cluster_centers = ms.cluster_centers_
# labels_unique = np.unique(labels)
# n_clusters_ = len(labels_unique)
# print("number of estimated clusters : %d" % n_clusters_)
# print(cluster_centers)
cc = ms.cluster_centers_
cc

array([[  40.71739277,  -73.98903854],
       [  33.4492317 , -112.00175051],
       [  41.87816072,  -87.62982257],
       ..., 
       [  10.473523  ,  -84.0167423 ],
       [  31.8023831 ,  -97.0916692 ],
       [  25.1497622 ,  -80.6826254 ]])

In [69]:
cl = ms.labels_

In [70]:
len(np.unique(cl))

3552

In [56]:
len(cc)

3552

In [138]:
of = [tuple(o) for o in np.array(read_csv(StringIO('''
33.751277, -118.188740, (Los Angeles)
25.867736, -80.324116, (Miami)
51.503016, -0.075479, (London)
52.378894, 4.885084, (Amsterdam)
39.366487, 117.036146, (Beijing)
-33.868457, 151.205134, (Sydney)
'''), header=None))[:,0:2]]
of

[(33.751277, -118.18874),
 (25.867735999999997, -80.324116),
 (51.503015999999995, -0.07547899999999999),
 (52.378894, 4.885084),
 (39.366487, 117.036146),
 (-33.868457, 151.20513400000002)]

In [68]:
from geopy.distance import vincenty
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(vincenty(newport_ri, cleveland_oh).meters)

866455.4329011001


In [67]:
import pickle
def pd(f, a):
    with open(f, 'wb') as handle:
        pickle.dump(a, handle)

def pl(f):
    with open(f, 'rb') as handle:
        return pickle.load(handle)
pd('ms.p', ms)

In [83]:
# np.array(Counter(cl).items())
# %%time
cl1 = list(cl)
%time cl2 = np.array([[i,cl1.count(i)] for i in set(cl)])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.27 ms
CPU times: user 10 s, sys: 50 ms, total: 10.1 s
Wall time: 10.1 s


In [94]:
cl3 = cl2[cl2[:,1] > 15]

In [107]:
vincenty((of[0,0], of[0,1]), (of[1,0], of[1,1])).meters

3741067.069343557

In [111]:
cl4 = cl3[:,0]

In [137]:
# cc1 = cc[cl4]
cc2 = [(i,tuple(cc[i])) for i in cl4]
# cc2

In [144]:
# v = lambda o,c: vincenty(tuple(o), tuple(c)).meters 
# %time b = [(v(o, c), i) for o in of for i,c in enumerate(cc1)]
b = [(vincenty(o, c).meters, i) for o in of for i,c in cc2]
b.sort()

In [151]:
# -33.8661460662 151.207082415
pf('m3-w1',pp(cc[b[0][1]]))

m3-w1="-33.8697913358 151.205630766"



In [152]:
pf('m3-w1-1',pp(cc[b[1][1]]))

m3-w1-1="52.3726819289 4.89260979211"

